# B. first automatic tag detection 

## Intro - Importing libraries and datasets

In [45]:
# import of libraries
import pandas as pd
from fuzzywuzzy import fuzz
from tqdm import tqdm

In [2]:
# import of thesaurus
thesaurus = pd.read_csv('data/thesaurus_key_words.csv', encoding="ISO-8859-1", sep=';')
thesaurus.head()

,classification_E,catégorie,symptome-fr,symptome-en,CIM_10,CIM11,Orphanet
0,E1,Période néonatale,Encéphalopathie myoclonique précoce,Benign familial neonatal epilepsy (BFNE),G40.8,8A61.0Y,1935.0
1,E2,Période néonatale,Epilepsie néonatale familiale bénigne (BFNE),Early myoclonic encephalopathy (EME),G40.8,8A61.10,1949.0
2,E3,Période néonatale,Syndrome d'ohtahara,Ohtahara syndrome,G40.8,8A62.Y,1934.0
3,E31,Nourrisons,Encépahlopathie myoclonique des affections non...,Myoclonic encephalopathy in nonprogressive dis...,G40.4,8A62.Y,86913.0
4,E33,Nourrisons,Epilepsie benigne du nourisson,Benign infantile epilepsy,G40.3,8A61.1Z,166302.0


In [3]:
# import of classification dataset
classification_dataset = pd.read_csv('data/classification_dataset.csv')
classification_dataset.head()

,filepath,report
0,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,Description: 2.5 to 5 hz spike/wave and polys...
1,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,LENGTH OF THE RECORDING: 22 minutes and 53 s...
2,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."
3,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,CLINICAL HISTORY: 27 year old gentleman with...
4,CR_Patients_info_patients-v0_4/edf/dev/01_tcp_...,"MEDICATIONS: Vimpat, Norvasc, Felbamate, Car..."


# I - Working with Levenshtein distance

In [79]:

# We will calculate the partial_ratio for each thesaury therme and update it in a result dataset

%%time

for i in tqdm(list(thesaurus['symptome-en'])):
    classification_dataset[i] = classification_dataset['report'].apply(lambda x: fuzz.partial_ratio(x, i)) 

df_results = pd.DataFrame(data=classification_dataset.columns[4:], columns=['target'])
df_results['ratio'] = df_results['target'].apply(lambda x: max(classification_dataset[x]))

# What can we predict at best?
df_results.sort_values(by='ratio', ascending=False)

100%|██████████| 46/46 [00:47<00:00,  1.03s/it]CPU times: user 47 s, sys: 46.9 ms, total: 47.1 s
Wall time: 47.3 s



,target,ratio
13,Lennox-Gastaut syndrome,100
16,Epilepsy with generalized tonicclonic seizure...,81
7,West syndrome,77
31,temporal epilepsy,76
23,central epilepsy,75
26,frontal epilepsy,75
27,insular epilepsy,75
6,Dravet syndrome,73
2,Benign infantile epilepsy,72
30,parietal epilepsy,71


In [133]:
# For a target, output the related reports sorted by partial_ratio

def research_similarity(target):
    df = pd.DataFrame(classification_dataset['report'])
    df['partial_ratio'] = df['report'].apply(lambda x: fuzz.partial_ratio(x, target))
    df = df.sort_values(by='partial_ratio', ascending=False)
    return df

' CLINICAL HISTORY: This is a 29-year-old, mentally retarded female with seizures since age 7, after removal of an occipital tumor.  Last seizure one week ago. MEDICATIONS: Dilantin. INTRODUCTION: Digital video EEG is performed in the lab using standard 10-20 system of electrode placement with one channel of EKG. Hyperventilation and photic stimulation were not performed.  This is an awake and light sleep record. DESCRIPTION OF THE RECORD: The background EEG is diffusely slow with theta activity and disorganized.  A rare, slow alpha rhythm of 7 Hz, 20 V is seen. Low voltage, frontocentral beta activity is also seen.  Soon after the EEG starts, bursts of 2 to 4 Hz frontally predominant sharp and slow wave complexes lasting for one to two seconds are seen.  In addition, frequent right focal spike and wave complexes are seen in the right hemisphere, mostly in the temporal region, but also in the occipital area.  This focal spike and wave activity occasionally generalized. Drowsiness is ch

## A - Focus on Lennox-Gastaut

In [137]:

research_similarity('Lennox-Gastaut')['partial_ratio'].head(30)

838     100
117     100
45      100
46      100
47      100
48      100
49      100
51      100
1258    100
843     100
220     100
1315    100
372     100
44      100
1107    100
276     100
1302    100
227     100
1211    100
547     100
1167    100
684     100
1269    100
597      50
596      50
595      50
594      50
593      50
598      50
809      43
Name: partial_ratio, dtype: int64

In [140]:
research_similarity('Lennox-Gastaut').report[227]

' CLINICAL HISTORY: This is a 27-year-old male with a history of severe MR, multiple medical problems with multiple brief seizures per month.  Seizures characterized by generalized shaking lasting 20 seconds. MEDICATIONS: Lamictal, Tegretol, Tranxene, and many others. INTRODUCTION: Digital video EEG is performed in the lab using standard 10-20 system of electrode placement with one channel of EKG. The patient is drowsy or somnolent. Photic stimulation is performed. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal and is primarily a mixture of rhythmic 3 Hz activity with smaller amounts of 2 Hz activity and some 4 to 5 Hz theta.  There are multifocal spike and slow wave complexes identified in the record including bifrontal, high amplitude spike and slow wave complexes with an approximately 2 Hz after going slow wave.  Focal epileptiform activity is also seen in the occipital regions, sometimes maximum at O2 and at other times with a poly spike wave component at O1-O2.

it works with 100 partial ratio

In [141]:
research_similarity('Lennox-Gastaut').report[597]

' History: The patient is a 28 year old man with a long-standing history of medically refractory epilepsy. Patient is admitted to continuous audio-video EEG monitoring for event characterization and potential seizure localization and  lateralization as a part of presurgical evaluation for potential epilepsy surgery. Seizure semiology: Type 1: No Aura. Head shaking side-to-side.  Stares blankly, loss of awareness, does not respond to stimuli during the event. Lasts about 1O min, occasionally eyes roll backwards. Type 2: Type 1with shaking of the arms and legs associated with the staring for about 2 min. Post event confusion for 30min. Admission  Medications: Lamictal 200 mg BID Zonisamide 300 mg qHS Conditions of the recording: A routine 21 channel digital EEG was recorded at the onset of monitoring. The patient then underwent continuous CCTV/EEG monitoring using a cable telemetry system. Scalp electrodes were placed and secured with collodion according to the 10-20 international electr

It's not working on 50 partial ratio.

## B - Focus on temporal epilepsy

In [154]:
research_similarity('temporal epilepsy')

,report,partial_ratio
377,EEG REMARKS: 7 L temporal Spikes but seems se...,76
191,CLINICAL HISTORY: 40 year old right handed ma...,71
194,CLINICAL HISTORY: 40 year old right handed ma...,71
1133,CLINICAL HISTORY: \tForty-seven-year-old male...,71
523,HISTORY: A 62-year-old woman with adult-onse...,71
...,...,...
145,"CLINICAL HISTORY: A 25-year-old man, with hi...",0
426,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,0
118,CLINICAL HISTORY: A 35-year-old woman with c...,0
688,REASON FOR STUDY: Change in mental status. C...,0


In [155]:
research_similarity('temporal epilepsy')['report'][191]

' CLINICAL HISTORY: 40 year old right handed male with encephalitis and recurrent seizures. MEDICATIONS: Lacosamide, dilantin, Ativan, Klonopin INTRODUCTION: Continuous digital video EEG monitoring was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. As this section of the records begins, the patient reports "he is feeling great" as if he is not having more seizures. Then subsequently he has 2 events that he describes as auras, which are seizures with impairment of awareness. He does have occasional myoclonic jerks. DESCRIPTION OF THE RECORD: This section of the 24-hour period includes more of the rhythmic repetitive slowing than noted at other times. Isolated high amplitude right hemispheric spike and wave activity is observed. Push button times include 5:20 which includes actually a seizure. Although the patient describes this as an aura, it is really a focal motor seizure with loss of axial tone and stiffening of the right leg. The patie

## B - Other research

In [171]:
research_similarity('Keppra Ativan famotidine Lovenox topiramate Flagyl Depakote').head(30)

,report,partial_ratio
44,DURATION OF STUDY: Study date 03/26/2013 thr...,90
51,DURATION OF STUDY: Study date 03/26/2013 thr...,90
48,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,90
47,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,90
46,DURATION OF STUDY: Study date 03/26/2013 thr...,90
45,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,90
52,REASON FOR STUDY: Seizures. CLINICAL HISTORY...,54
652,CLINICAL HISTORY: 60 year old right handed fe...,53
1071,CLINICAL HISTORY: 60 year old right handed fe...,53
1070,CLINICAL HISTORY: 60 year old right handed fe...,53


## Using rake

In [176]:
research_similarity('temporal epilepsy')['report'][191]

' CLINICAL HISTORY: 40 year old right handed male with encephalitis and recurrent seizures. MEDICATIONS: Lacosamide, dilantin, Ativan, Klonopin INTRODUCTION: Continuous digital video EEG monitoring was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. As this section of the records begins, the patient reports "he is feeling great" as if he is not having more seizures. Then subsequently he has 2 events that he describes as auras, which are seizures with impairment of awareness. He does have occasional myoclonic jerks. DESCRIPTION OF THE RECORD: This section of the 24-hour period includes more of the rhythmic repetitive slowing than noted at other times. Isolated high amplitude right hemispheric spike and wave activity is observed. Push button times include 5:20 which includes actually a seizure. Although the patient describes this as an aura, it is really a focal motor seizure with loss of axial tone and stiffening of the right leg. The patie

In [173]:
from rake_nltk import Rake

r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.

r.extract_keywords_from_text(research_similarity('temporal epilepsy')['report'][191])

r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.

['isolated high amplitude right hemispheric spike',
 '40 year old right handed male',
 'characteristic tonic clonic activity obscures',
 'push button times include 5',
 'continuous digital video eeg monitoring',
 'high amplitude sharp waves',
 'particularly parietal occipital region',
 'rhythmic mixed theta delta',
 'bedside using standard 10',
 'prominent sharp waves',
 'additional sharp waves',
 'previous eeg monitoring',
 'occasional myoclonic jerks',
 'localization related mechanism',
 'little bit confused',
 'blood pressure cuff',
 'eeg monitor ends',
 'third seizure type',
 'hour period includes',
 'focal motor seizure',
 'rhythmic repetitive slowing',
 'complex partial seizures',
 'seizure time 13',
 'eeg monitoring',
 'right leg',
 'wave activity',
 'rhythmic slowing',
 'rhythmic discharge',
 'entire eeg',
 'includes actually',
 'recurrent seizures',
 'records begins',
 'movement artifact',
 'left hemisphere',
 'left hand',
 'klonopin introduction',
 'feeling great',
 'electrod

Keppra, Ativan, famotidine, Lovenox, topiramate, Flagyl, Depakote, and Vimpat.
Carbatrol, Topamax
Keppra, Ativan, famotidine, Lovenox, topiramate, Flagyl, Depakote, and Vimpat
Keppra, Ativan, famotidine, Lovenox, topiramate, Flagyl, Depakote

In [180]:
research_similarity('temporal epilepsy')['report'][191]

' CLINICAL HISTORY: 40 year old right handed male with encephalitis and recurrent seizures. MEDICATIONS: Lacosamide, dilantin, Ativan, Klonopin INTRODUCTION: Continuous digital video EEG monitoring was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. As this section of the records begins, the patient reports "he is feeling great" as if he is not having more seizures. Then subsequently he has 2 events that he describes as auras, which are seizures with impairment of awareness. He does have occasional myoclonic jerks. DESCRIPTION OF THE RECORD: This section of the 24-hour period includes more of the rhythmic repetitive slowing than noted at other times. Isolated high amplitude right hemispheric spike and wave activity is observed. Push button times include 5:20 which includes actually a seizure. Although the patient describes this as an aura, it is really a focal motor seizure with loss of axial tone and stiffening of the right leg. The patie

## New optic: doing by sentence

In [179]:
fuzz.partial_ratio('He does have occasional myoclonic jerks', 'myoclonic')

100

In [186]:
fuzz.partial_ratio('myoclonic', research_similarity('temporal epilepsy')['report'][191])

33

In [309]:
def partial_ratio_by_sentence(texte, target):
    max = 0
    for i in texte.split('.'):
        if fuzz.partial_ratio(i, target) > max:
            if len(i) > 5:
                max = fuzz.partial_ratio(i, target)
    return max


In [312]:
partial_ratio_by_sentence(classification_dataset.report.iloc[191] , 'myoclonic')

100

In [313]:
texte = classification_dataset_by_sentence['report'][1]
partial_ratio_by_sentence(texte, 'myoclonic')


56

In [306]:
# For a target, output the related reports sorted by partial_ratio

def research_similarity_by_sentence(target):
    df = pd.DataFrame(classification_dataset['report'])
    df['partial_ratio'] = df['report'].apply(lambda x: partial_ratio_by_sentence(x, target))
    df = df.sort_values(by='partial_ratio', ascending=False)
    return df

In [307]:

# We will calculate the partial_ratio for each thesaury therme and update it in a result dataset
%time

classification_dataset_by_sentence = classification_dataset[['filepath', 'report']]

for i in tqdm(list(thesaurus['symptome-en'])[1:2]):
    print(i)
    classification_dataset_by_sentence[i] = classification_dataset_by_sentence['report'].apply(lambda x: partial_ratio_by_sentence(x, i)) 

df_results_by_sentence = pd.DataFrame(data=classification_dataset_by_sentence.columns[2:], columns=['target'])
df_results_by_sentence['ratio'] = df_results_by_sentence['target'].apply(lambda x: classification_dataset_by_sentence[x].max())

# What can we predict at best?
df_results_by_sentence.sort_values(by='ratio', ascending=False)

0%|          | 0/1 [00:00<?, ?it/s]CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.3 µs
Early myoclonic encephalopathy (EME)
100%|██████████| 1/1 [01:06<00:00, 67.00s/it]


,target,ratio
0,Early myoclonic encephalopathy (EME),70


In [311]:
for i in range(5):
    display(classification_dataset_by_sentence.sort_values(by='Early myoclonic encephalopathy (EME)', ascending=False)['report'].iloc[0])

' CLINICAL HISTORY: 61 year old left handed gentleman with a right MCA infarct,seizures characterized by right-sided twitching, acute fever, encephalopathy, critical illness. MEDICATIONS: Dilantin, Lamotrigine, Digoxin, Zocor, Flomax, Citalopram, ASA, Risperidone, Miralax, Protonix, Baclofen, Seroquel, Aldactone, Midodrine, Vancomycin, Lasix, Dobutamine INTRODUCTION: Digital video EEG was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. The patient is febrile and poorly responsive. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal. The overall pattern is a mixture of theta and delta. The previously described asymmetry is not as prominent in terms of disruption of activity from the right hemisphere. There are frontally predominant triphasic waves which are asymmetric, left more so than right. There are also left temporal and frontal sharp waves noted intermittently. They have a periodic nature to them but do not evolve. On o

' CLINICAL HISTORY: 61 year old left handed gentleman with a right MCA infarct,seizures characterized by right-sided twitching, acute fever, encephalopathy, critical illness. MEDICATIONS: Dilantin, Lamotrigine, Digoxin, Zocor, Flomax, Citalopram, ASA, Risperidone, Miralax, Protonix, Baclofen, Seroquel, Aldactone, Midodrine, Vancomycin, Lasix, Dobutamine INTRODUCTION: Digital video EEG was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. The patient is febrile and poorly responsive. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal. The overall pattern is a mixture of theta and delta. The previously described asymmetry is not as prominent in terms of disruption of activity from the right hemisphere. There are frontally predominant triphasic waves which are asymmetric, left more so than right. There are also left temporal and frontal sharp waves noted intermittently. They have a periodic nature to them but do not evolve. On o

' CLINICAL HISTORY: 61 year old left handed gentleman with a right MCA infarct,seizures characterized by right-sided twitching, acute fever, encephalopathy, critical illness. MEDICATIONS: Dilantin, Lamotrigine, Digoxin, Zocor, Flomax, Citalopram, ASA, Risperidone, Miralax, Protonix, Baclofen, Seroquel, Aldactone, Midodrine, Vancomycin, Lasix, Dobutamine INTRODUCTION: Digital video EEG was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. The patient is febrile and poorly responsive. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal. The overall pattern is a mixture of theta and delta. The previously described asymmetry is not as prominent in terms of disruption of activity from the right hemisphere. There are frontally predominant triphasic waves which are asymmetric, left more so than right. There are also left temporal and frontal sharp waves noted intermittently. They have a periodic nature to them but do not evolve. On o

' CLINICAL HISTORY: 61 year old left handed gentleman with a right MCA infarct,seizures characterized by right-sided twitching, acute fever, encephalopathy, critical illness. MEDICATIONS: Dilantin, Lamotrigine, Digoxin, Zocor, Flomax, Citalopram, ASA, Risperidone, Miralax, Protonix, Baclofen, Seroquel, Aldactone, Midodrine, Vancomycin, Lasix, Dobutamine INTRODUCTION: Digital video EEG was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. The patient is febrile and poorly responsive. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal. The overall pattern is a mixture of theta and delta. The previously described asymmetry is not as prominent in terms of disruption of activity from the right hemisphere. There are frontally predominant triphasic waves which are asymmetric, left more so than right. There are also left temporal and frontal sharp waves noted intermittently. They have a periodic nature to them but do not evolve. On o

' CLINICAL HISTORY: 61 year old left handed gentleman with a right MCA infarct,seizures characterized by right-sided twitching, acute fever, encephalopathy, critical illness. MEDICATIONS: Dilantin, Lamotrigine, Digoxin, Zocor, Flomax, Citalopram, ASA, Risperidone, Miralax, Protonix, Baclofen, Seroquel, Aldactone, Midodrine, Vancomycin, Lasix, Dobutamine INTRODUCTION: Digital video EEG was performed at bedside using standard 10-20 system of electrode placement with 1 channel of EKG. The patient is febrile and poorly responsive. DESCRIPTION OF THE RECORD: The background EEG is markedly abnormal. The overall pattern is a mixture of theta and delta. The previously described asymmetry is not as prominent in terms of disruption of activity from the right hemisphere. There are frontally predominant triphasic waves which are asymmetric, left more so than right. There are also left temporal and frontal sharp waves noted intermittently. They have a periodic nature to them but do not evolve. On o

In [295]:
def partial_ratio_by_sentence_bis(texte, target):
    max = 0
    for i in texte.split('.'):
        if fuzz.partial_ratio(i, target) > max:
            if len(i) > 10:
                print(i)
                max = fuzz.partial_ratio(i, target)
                print(max)
    return max


In [293]:
texte = classification_dataset_by_sentence['report'][1]

In [296]:
partial_ratio_by_sentence_bis(texte, 'Early myoclonic encephalopathy (EME)')

LENGTH OF THE RECORDING:  22 minutes and 53 seconds CLINICAL HISTORY:  The patient is a 27-year-old man with past medical history of cognitive delay, mitochondrial disorder, seizure disorder
33
 MEDICATIONS:  Ativan, Dilantin, clonazepam, chlorpromazine, amlodipine
39
  The majority of the background, however, consists of low amplitude fast activity admixed with polymorphic frequently sharply contoured central theta and delta activity
44


44